In [ ]:
root_dir = !git rev-parse --show-toplevel
import os; os.chdir(root_dir[0])

In [ ]:
import joblib
import pandas as pd

municipalities = joblib.load('data/export/municipality.pickle')
udh = joblib.load('data/export/udh.pickle')

udh['level'] = 'udh'
municipalities['level'] = 'municipality'

udh.columns = [c.lower() for c in udh.columns]
municipalities.columns = [c.lower() for c in municipalities.columns]


municipalities = municipalities.to_crs({'init': 'epsg:4326'})
udh = udh.to_crs({'init': 'epsg:4326'})

municipalities.sindex
udh.sindex;

In [ ]:
columns_descriptions = pd.read_excel('data/preprocessed/A - DICIONÁRIO dos indicadores do Atlas.xlsx')
columns_descriptions = columns_descriptions.set_index('SIGLA')
columns_descriptions.index.names = [None]
columns_descriptions = columns_descriptions.T
columns_descriptions.columns = [c.lower() for c in columns_descriptions.columns]
columns_descriptions #.to_dict()

In [ ]:
columns_descriptions.to_dict();

In [ ]:
from typing import Tuple, List
from shapely.geometry import Point, MultiPoint
import geopandas as gpd

In [ ]:
def get_info(points: List[Point], level='best'):
    if level not in ('best', 'all'):
        raise TypeError('choose a suported_level')
    points = [Point(p) for p in points]
    points =  gpd.GeoDataFrame(geometry=points)
    points.crs = {'init' :'epsg:4326'} # default to this projection
    udh_level = gpd.sjoin(points, udh)
    missing_points = points.index.difference(udh_level.index)
    if level == 'best':
        municipality_level = gpd.sjoin(points.loc[missing_points], municipalities)
        out = pd.concat([udh_level, municipality_level], sort=False)
        assert out.index.shape == points.index.shape
        return out.loc[points.index]
    elif level == 'all':
        raise NotImplementedError


In [ ]:
test = [Point(-59.23352, -3.35030), Point(-60.17875, -3.27442), Point(-43.198976, -22.956012), Point(-43.194284, -22.947361)] * 1000
df = get_info(test[0:2])
df.dropna(axis=1).espvida

In [ ]:
df.columns.difference(columns_descriptions.columns)
columns_descriptions.columns.difference(df.columns)

In [ ]:
df.columns.tolist();

In [ ]:
%%time
ble = get_info(test)
ble.head()

In [ ]:
ble.rdpc10

# Gutter

## Metadata in dataframes (comments columns)

In [ ]:
def comment_column(self, column, comment):
    assert column in self.columns
    self._extra_metadata = getattr(self, '_extra_metadata', {})
    self._extra_metadata[column] = comment
    
def bind(instance, func, as_name=None):
    """
    Bind the function *func* to *instance*, with either provided name *as_name*
    or the existing name of *func*. The provided *func* should accept the 
    instance as the first argument, i.e. "self".
    """
    if as_name is None:
        as_name = func.__name__
    bound_method = func.__get__(instance, instance.__class__)
    setattr(instance, as_name, bound_method)
    return bound_method

ble.comment_column = bind(ble, comment_column)

In [ ]:
ble.comment_column('Cod_ID', 'comentario')